**Download Dataset Karate and processing**

In [1]:
import pandas as pd 
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
#ffmpeg_extract_subclip(input_video_path, t1, t2, targetname=output_video_path)

In [2]:
import os  
from os.path import join
from moviepy.editor import VideoFileClip
import time

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3186688/45929032 bytes (6.9%)7086080/45929032 bytes (15.4%)11108352/45929032 bytes (24.2%)14868480/45929032 bytes (32.4%)18284544/45929032 bytes (39.8%)22200320/45929032 bytes (48.3%)26083328/45929032 bytes (56.8%)30081024/45929032 bytes (65.5%)34045952/45929032 bytes (74.1%)38019072/45929032 bytes (82.8%)41934848/45929032 bytes (91.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

In [3]:
import os ,re 
from os.path import *
from os import listdir,mkdir
from glob import glob

def video_files_in_folder(folder):
    return [os.path.join(folder, f) for f in os.listdir(folder) if re.match(r'.*\.(jpg|jpeg|png|mp4)', f, flags=re.I)]

def get_videos_from_paths(data_dir ,type='one dir'):
    if type == 'one dir':
        return video_files_in_folder(data_dir) 
    else:
        list_pp  = []
        for class_dir in listdir(data_dir):
            if not isdir(join(data_dir, class_dir)):
              continue

            for vid_path in video_files_in_folder(join(data_dir, class_dir)):
                list_pp.append(vid_path) 
        return list_pp
        

In [ ]:
root_input_data  = '/content/drive/MyDrive/Show_Karate'
path_output_data = '/content/Show_Karate'

list_dir = glob(root_input_data+"/*/")
count = 0
for i ,dir in enumerate(list_dir):
    path_out = join(path_output_data, dir.split('/')[-2])
    if not os.path.isdir(path_out):
       os.makedirs(path_out)

    for item_vi in get_videos_from_paths(dir ,type="Tow dir"):
        name = item_vi.split('/')[-1].replace('.mp4' ,'{}_.mp4'.format(count)) 
        try:
            file_path_in  = item_vi
            file_path_out = join(path_out ,name)
            !ffmpeg -i '{file_path_in}' -vcodec mpeg4 -s 340x256 -b 300k -r 30 -acodec libmp3lame -ar 48000 -ab 112k '{file_path_out}' -y
            
        except:
           continue
        count+=1

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
    comment         : Create videos with https://clipchamp.com/en/video-editor - free online video editor, video compressor, video converter.
  Duration: 00:00:01.83, start: 0.000000, bitrate: 831 kb/s
    Stream #0:0(und): Video: h264 (Constrained Baseline) (avc1 / 0x31637661), yuv420p, 852x480, 815 kb/s, 30 fps, 30 tbr, 15360 tbn, 60 tbc (default)
    Metadata:
      handler_name    : VideoHandler
Please use -b:a or -b:v, -b is ambiguous
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> mpeg4 (native))
Press [q] to stop, [?] for help
Output #0, mp4, to '/content/Show_Karate/Tseuki_Jodan/Tseuki_Jodan_652_.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    comment         : Create videos with https://clipchamp.com/en/video-editor - free online video editor, video compressor, video converter.
    encoder         : Lavf57.83.100
    Stream #0

**Extra Pose and normalize data & [Save as csv]**

In [ ]:
!gdown https://drive.google.com/uc?id=1Fcw6P8qs4wNFXm9HTZ_3fzpCzSBgHjsK   
!unzip -qq Fastpose_st_gcn.zip 
%cd Fastpose_st_gcn 

In [ ]:
import os
import cv2
import time
import torch
import pandas as pd
import numpy as np
import torchvision.transforms as transforms

from DetectorLoader import TinyYOLOv3_onecls
from PoseEstimateLoader import SPPE_FastPose
from fn import vis_frame_fast

In [ ]:
import os ,re 
from os.path import *
from os import listdir,mkdir

def video_files_in_folder(folder):
    return [os.path.join(folder, f) for f in os.listdir(folder) if re.match(r'.*\.(jpg|jpeg|png|mp4)', f, flags=re.I)]

def get_videos_from_paths(data_dir ,type='one dir'):
    if type == 'one dir':
        return video_files_in_folder(data_dir) 
    else:
        list_pp  = []
        for class_dir in listdir(data_dir):
            if not isdir(join(data_dir, class_dir)):
              continue

            for vid_path in video_files_in_folder(join(data_dir, class_dir)):
                list_pp.append(vid_path) 
        return list_pp
        

In [ ]:
save_path  = '/content/drive/MyDrive/E-sport_final/Karate_output/Show_Karate_pose_score.csv'

# DETECTION MODEL.
detector = TinyYOLOv3_onecls()

# POSE MODEL.
#340x256
inp_h = 340
inp_w = 256
pose_backbone='resnet50'
pose_estimator = SPPE_FastPose(pose_backbone ,inp_h, inp_w)

# with score.
columns = ['video', 'frame', 'Nose_x', 'Nose_y', 'Nose_s', 'LShoulder_x', 'LShoulder_y', 'LShoulder_s',
           'RShoulder_x', 'RShoulder_y', 'RShoulder_s', 'LElbow_x', 'LElbow_y', 'LElbow_s', 'RElbow_x',
           'RElbow_y', 'RElbow_s', 'LWrist_x', 'LWrist_y', 'LWrist_s', 'RWrist_x', 'RWrist_y', 'RWrist_s',
           'LHip_x', 'LHip_y', 'LHip_s', 'RHip_x', 'RHip_y', 'RHip_s', 'LKnee_x', 'LKnee_y', 'LKnee_s',
           'RKnee_x', 'RKnee_y', 'RKnee_s', 'LAnkle_x', 'LAnkle_y', 'LAnkle_s', 'RAnkle_x', 'RAnkle_y',
           'RAnkle_s', 'label']

def normalize_points_with_size(points_xy, width, height, flip=False):
            points_xy[:, 0] /= width
            points_xy[:, 1] /= height
            if flip:
                points_xy[:, 0] = 1 - points_xy[:, 0]
            return points_xy

def extra_pose(list_vid ,label):
        for vid in list_vid:
            print(f'Process on: {vid}')
            df = pd.DataFrame(columns=columns)
            cur_row = 0
            
            cap = cv2.VideoCapture(vid)
            frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
                          int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

            # Bounding Boxs Labels.
            annot = None
            fps_time = 0
            i = 1
            while True:
                ret, frame = cap.read()
                if ret:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # Pose Labels.
                    cls_idx = label

                    if annot:
                        bb = np.array(annot.iloc[i-1, 2:].astype(int))
                    else:
                        bb = detector.detect(frame)
                    if bb is None:
                      continue
                    bb = bb[0, :4].numpy().astype(int)
                    bb[:2] = np.maximum(0, bb[:2] - 5)
                    bb[2:] = np.minimum(frame_size, bb[2:] + 5) if bb[2:].any() != 0 else bb[2:]

                    result = []
                    if bb.any() != 0:
                        result = pose_estimator.predict(frame, torch.tensor(bb[None, ...]),
                                                        torch.tensor([[1.0]]))

                    if len(result) > 0:
                        pt_norm = normalize_points_with_size(result[0]['keypoints'].numpy().copy(),
                                                            frame_size[0], frame_size[1])
                        pt_norm = np.concatenate((pt_norm, result[0]['kp_score']), axis=1)

                        #idx = result[0]['kp_score'] <= 0.05
                        #pt_norm[idx.squeeze()] = np.nan
                        row = [vid, i, *pt_norm.flatten().tolist(), cls_idx]
                        scr = result[0]['kp_score'].mean()
                    else:
                        row = [vid, i, *[np.nan] * (13 * 3), cls_idx]
                        scr = 0.0

                    df.loc[cur_row] = row
                    cur_row += 1

                    # VISUALIZE.
                    frame = vis_frame_fast(frame, result)
                    frame = cv2.rectangle(frame, (bb[0], bb[1]), (bb[2], bb[3]), (0, 255, 0), 2)
                    frame = cv2.putText(frame, 'Frame: {}, Pose: {}, Score: {:.4f}'.format(i, cls_idx, scr),
                                        (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    frame = frame[:, :, ::-1]
                    fps_time = time.time()
                    i += 1

                else:
                    break

            cap.release() 

            if os.path.exists(save_path):
                df.to_csv(save_path, mode='a', header=False, index=False)
            else:
                df.to_csv(save_path, mode='w', index=False)

        return df

Loading pose model from ./Models/sppe/fast_res50_256x192.pth


In [ ]:
Gidan_barei    = get_videos_from_paths('/content/Show_Karate/Gidan_barei')
Mae_giri      = get_videos_from_paths('/content/Show_Karate/Mae_giri')
Mawashi_giri = get_videos_from_paths('/content/Show_Karate/Mawashi_giri')
Shuto_uke     = get_videos_from_paths('/content/Show_Karate/Shuto_uke')
Soto_uke           = get_videos_from_paths('/content/Show_Karate/Soto_uke')
Tseuki_Jodan      = get_videos_from_paths('/content/Show_Karate/Tseuki_Jodan')
Tseuki_chudan  = get_videos_from_paths('/content/Show_Karate/Tseuki_chudan')
Yoko_giri  = get_videos_from_paths('/content/Show_Karate/Yoko_giri')


In [ ]:
[Gidan_barei ,Mae_giri ,Mawashi_giri ,Shuto_uke ,Soto_uke ,Tseuki_Jodan ,Tseuki_chudan ,Yoko_giri]

[['/content/Show_Karate/Gidan_barei/Gidan_barei_3119_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_7130_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_6122_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_2126_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_2135_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_2118_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_9125_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_7140_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_7123_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_4120_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_5129_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_1117_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_3136_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_3127_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_5121_.mp4',
  '/content/Show_Karate/Gidan_barei/Gidan_barei_8133_.mp4',
  '/content/Show_Karate/Gidan_barei/Gida

In [ ]:
extra_pose(Gidan_barei ,0) #fall 
extra_pose(Mae_giri ,1) #fall 
extra_pose(Mawashi_giri ,2) #fall 
extra_pose(Shuto_uke ,3) #fall 
extra_pose(Soto_uke ,4) #fall 
extra_pose(Tseuki_Jodan ,5) #fall 
extra_pose(Tseuki_chudan ,6) #fall 
extra_pose(Yoko_giri ,7) #fall 
 

Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_3119_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_7130_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_6122_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_2126_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_2135_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_2118_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_9125_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_7140_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_7123_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_4120_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_5129_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_1117_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_3136_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_3127_.mp4
Process on: /content/Show_Karate/Gidan_barei/Gidan_barei_5121_

,video,frame,Nose_x,Nose_y,Nose_s,LShoulder_x,LShoulder_y,LShoulder_s,RShoulder_x,RShoulder_y,RShoulder_s,LElbow_x,LElbow_y,LElbow_s,RElbow_x,RElbow_y,RElbow_s,LWrist_x,LWrist_y,LWrist_s,RWrist_x,RWrist_y,RWrist_s,LHip_x,LHip_y,LHip_s,RHip_x,RHip_y,RHip_s,LKnee_x,LKnee_y,LKnee_s,RKnee_x,RKnee_y,RKnee_s,LAnkle_x,LAnkle_y,LAnkle_s,RAnkle_x,RAnkle_y,RAnkle_s,label
0,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,1,0.448813,0.344891,0.981318,0.442686,0.393710,0.802407,0.418181,0.401847,0.882608,0.479444,0.450666,0.782188,0.436560,0.466939,0.858688,0.503950,0.418120,0.879524,0.461065,0.426256,0.896214,0.454939,0.523895,0.697348,0.442686,0.523895,0.684314,0.442686,0.629670,0.697438,0.436560,0.629670,0.765314,0.430434,0.751718,0.772144,0.430434,0.751718,0.842609,7
1,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,2,0.448813,0.344891,0.983148,0.442686,0.393710,0.798639,0.418181,0.401847,0.887926,0.479444,0.450666,0.767522,0.436560,0.466939,0.859090,0.503950,0.409983,0.866167,0.461065,0.426256,0.886522,0.454939,0.523895,0.659741,0.442686,0.532031,0.671173,0.442686,0.629670,0.684798,0.436560,0.629670,0.735248,0.430434,0.751718,0.803762,0.430434,0.751718,0.769828,7
2,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,3,0.448813,0.344891,0.984228,0.442686,0.393710,0.799558,0.418181,0.401847,0.884855,0.479444,0.450666,0.766242,0.436560,0.466939,0.861468,0.503950,0.409983,0.845983,0.461065,0.426256,0.907198,0.454939,0.523895,0.658742,0.442686,0.532031,0.674354,0.442686,0.637806,0.688234,0.436560,0.629670,0.743935,0.430434,0.751718,0.807598,0.430434,0.751718,0.794495,7
3,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,4,0.448894,0.345517,0.984562,0.448894,0.394682,0.801239,0.418045,0.402876,0.878339,0.479743,0.452042,0.829957,0.436554,0.468430,0.885017,0.504422,0.411071,0.889119,0.461234,0.427459,0.924246,0.455064,0.525790,0.649238,0.442724,0.525790,0.661140,0.442724,0.632315,0.680780,0.436554,0.632315,0.715936,0.430385,0.747035,0.840263,0.430385,0.747035,0.641451,7
4,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,5,0.450365,0.345517,0.977619,0.450365,0.394682,0.813046,0.419516,0.402876,0.888328,0.481214,0.443848,0.852403,0.431855,0.468430,0.864124,0.505893,0.411071,0.920265,0.456534,0.427459,0.908044,0.456534,0.525790,0.698468,0.438025,0.525790,0.712157,0.444195,0.632315,0.724141,0.438025,0.632315,0.746435,0.431855,0.747035,0.848955,0.431855,0.747035,0.684044,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,70,0.447586,0.347266,0.946183,0.428816,0.388814,0.787778,0.422559,0.397124,0.904238,0.441329,0.455291,0.594303,0.428816,0.480220,0.701766,0.466356,0.471911,0.844453,0.460099,0.505149,0.787331,0.441329,0.530078,0.670960,0.441329,0.530078,0.717721,0.441329,0.638104,0.684750,0.441329,0.638104,0.730342,0.428816,0.746129,0.751691,0.422559,0.746129,0.807654,7
70,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,71,0.449219,0.344727,0.950130,0.430188,0.395277,0.780114,0.423845,0.395277,0.905858,0.442876,0.454252,0.709963,0.423845,0.479528,0.741086,0.468250,0.471103,0.861846,0.461906,0.504803,0.794461,0.442876,0.530078,0.701668,0.436532,0.530078,0.769462,0.442876,0.631179,0.691075,0.442876,0.631179,0.750238,0.423845,0.749130,0.736040,0.423845,0.749130,0.794304,7
71,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,72,0.445709,0.343639,0.953348,0.433630,0.391766,0.857717,0.427591,0.391766,0.930120,0.445709,0.447914,0.746002,0.427591,0.463956,0.708357,0.469867,0.479998,0.871310,0.457788,0.496040,0.789169,0.439670,0.528125,0.684580,0.439670,0.528125,0.705163,0.439670,0.632400,0.669658,0.439670,0.632400,0.717314,0.427591,0.752717,0.789982,0.427591,0.752717,0.815283,7
72,/content/Show_Karate/Yoko_giri/Yoko_giri_18_.mp4,73,0.446936,0.341726,0.933664,0.429209,0.388814,0.792680,0.429209,0.396662,0.932267,0.441027,0.459446,0.722886,0.441027,0.459446,0.768024,0.464663,0.490838,0.

**Smoothing & [save as pkl]**

In [ ]:
%cd /content/Fastpose_st_gcn

/content/Fastpose_st_gcn


In [ ]:
"""
This script to create dataset and labels by clean off some NaN, do a normalization,
label smoothing and label weights by scores.
"""
import os
import pickle
import numpy as np
import pandas as pd


class_names = ['Gidan_barei' ,'Mae_giri' ,'Mawashi_giri' ,'Shuto_uke' ,'Soto_uke' ,'Tseuki_Jodan' ,'Tseuki_chudan' ,'Yoko_giri']

main_parts = ['LShoulder_x', 'LShoulder_y', 'RShoulder_x', 'RShoulder_y', 'LHip_x', 'LHip_y',
              'RHip_x', 'RHip_y']
main_idx_parts = [1, 2, 7, 8, -1]  # 1.5

csv_pose_file = '/content/drive/MyDrive/E-sport_final/Karate_output/Show_Karate_pose_score.csv'
global save_path
save_path = '/content/drive/MyDrive/E-sport_final/Karate_output/Show_Karate_norm.pkl'

# Params.
smooth_labels_step = 8
n_frames = 30
skip_frame = 1

annot = pd.read_csv(csv_pose_file)


# Remove NaN.
idx = annot.iloc[:, 2:-1][main_parts].isna().sum(1) > 0
idx = np.where(idx)[0]
annot = annot.drop(idx)
# One-Hot Labels.
label_onehot = pd.get_dummies(annot['label'])
annot = annot.drop('label', axis=1).join(label_onehot)
cols = label_onehot.columns.values


def scale_pose(xy):
    """
    Normalize pose points by scale with max/min value of each pose.
    xy : (frames, parts, xy) or (parts, xy)
    """
    if xy.ndim == 2:
        xy = np.expand_dims(xy, 0)
    xy_min = np.nanmin(xy, axis=1)
    xy_max = np.nanmax(xy, axis=1)
    for i in range(xy.shape[0]):
        xy[i] = ((xy[i] - xy_min[i]) / (xy_max[i] - xy_min[i])) * 2 - 1
    return xy.squeeze()

def seq_label_smoothing(labels, max_step=10):
    steps = 0
    remain_step = 0
    target_label = 0
    active_label = 0
    start_change = 0
    max_val = np.max(labels)
    min_val = np.min(labels)
    for i in range(labels.shape[0]):
        if remain_step > 0:
            if i >= start_change:
                labels[i][active_label] = max_val * remain_step / steps
                labels[i][target_label] = max_val * (steps - remain_step) / steps \
                    if max_val * (steps - remain_step) / steps else min_val
                remain_step -= 1
            continue

        diff_index = np.where(np.argmax(labels[i:i+max_step], axis=1) - np.argmax(labels[i]) != 0)[0]
        if len(diff_index) > 0:
            start_change = i + remain_step // 2
            steps = diff_index[0]
            remain_step = steps
            target_label = np.argmax(labels[i + remain_step])
            active_label = np.argmax(labels[i])
    return labels


feature_set = np.empty((0, n_frames, 14, 3))
labels_set = np.empty((0, len(cols)))
vid_list = annot['video'].unique()


In [ ]:
for vid in vid_list:
        print(f'Process on: {vid}')
        data = annot[annot['video'] == vid].reset_index(drop=True).drop(columns='video')
        # Label Smoothing.
        esp = 0.1
        data[cols] = data[cols] * (1 - esp) + (1 - data[cols]) * esp / (len(cols) - 1)
        data[cols] = seq_label_smoothing(data[cols].values, smooth_labels_step)
        # Separate continuous frames.
        frames = data['frame'].values
        frames_set = []
        fs = [0]
        for i in range(1, len(frames)):
            if frames[i] < frames[i-1] + 10:
                fs.append(i)
            else:
                frames_set.append(fs)
                fs = [i]
        frames_set.append(fs)
        
        for fs in frames_set:
            xys = data.iloc[fs, 1:-len(cols)].values.reshape(-1, 13, 3)
            # Scale pose normalize.
            xys[:, :, :2] = scale_pose(xys[:, :, :2])
            # Add center point.
            xys = np.concatenate((xys, np.expand_dims((xys[:, 1, :] + xys[:, 2, :]) / 2, 1)), axis=1)

            # Weighting main parts score.
            scr = xys[:, :, -1].copy()
            scr[:, main_idx_parts] = np.minimum(scr[:, main_idx_parts] * 1.5, 1.0)
            # Mean score.
            scr = scr.mean(1)

            # Targets.
            lb = data.iloc[fs, -len(cols):].values
            # Apply points score mean to all labels.
            lb = lb * scr[:, None]

            for i in range(xys.shape[0] - n_frames):
                feature_set = np.append(feature_set, xys[i:i+n_frames][None, ...], axis=0)
                labels_set  = np.append(labels_set, lb[i:i+n_frames].mean(0)[None, ...], axis=0)

with open(save_path, 'wb') as f:
        pickle.dump((feature_set, labels_set), f) 


**Trainning**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')